## **ENGR 2900 Project 2 - Hand Pose Estimation**

**Introduction**

In this Project you will have the opportunity to build a 2D hand pose model and train it on the [Ego-Exo4D dataset](https://ego-exo4d-data.org/) to perform 2D hand pose estimation. The images you will be using are from first-person videos captured by [Project Aria glasses](https://www.projectaria.com/) worn by participants performing various human activities that are highly related to hand movements, e.g. cooking, playing music, bike repairing and covid testing. See Figure 1 for some examples.

<div style="text-align: center;">
    <div style="display: center; justify-content: space-between;">
        <img src="imgs/bike_repairing_sample.png" alt="Bike repairing" style="width: 24%;">
        <img src="imgs/cooking_sample.png" alt="Cooking" style="width: 24%;">
        <img src="imgs/covid_test_sample.png" alt="Covid test" style="width: 24%;">
        <img src="imgs/music_sample.png" alt="Music" style="width: 24%;">
    </div>
    <h1 style="font-size: 16px;">Figure 1: Sample images in Ego-Exo4D dataset</h1>
</div>

**Hand pose estimation**

The task of hand pose estimation is usually tackled by a top-down method, meaning a detector is first used to find the hand bounding box (where the hand is at within the original image). Then we get the cropped hand image and feed this into hand pose model as input to perform hand pose estimation. We will follow the top-down method in this project. See Figure 2 about the original image and cropped hand image.

<div style="text-align: center;">
    <div style="display: center; justify-content: space-between;">
        <img src="imgs/top_down_sample.png" alt="top_down" style="width: 80%;">
    </div>
    <h1 style="font-size: 16px;">Figure 2: Original image and cropped hand image</h1>
</div>

**What to do**

In this project, your main goal is to implement the second step - to develop and train a 2D hand pose estimation model which predicts the location of each hand joint given a hand image. A list of helper scripts from loading cropped hand image to model training have been provided. The parts that need your work are all within this notebook marked with **TODOS**. See below for Project 2 file structure and helper script introduction.

- `imgs/`: this is the directory where sample images for display are stored.
- `dataset/`
    - `dataset.py`: main Dataset to load and preprocess Ego-Exo4D data for model training.
    - `data_util.py`: a list of utility functions to help build the main Dataset, e.g. affine transformation to get cropped hand image.
    - `data_vis.py`: some helper functions to visualize the images with hand pose annotation.
- `utils/`
    - `functions.py`: a list of utility functions to help model training and debugging.
    - `loss.py`: implementation of loss function to supervise the model learning.
- `ENGR2900_Project2.ipynb`: intergrate every parts from above together from loading dataset to model training and testing. This is where you need to fill in your implementation.


### 0 - Set-up

Run cell below to load in necessary packages and helper scripts. Make sure you have set up and activate the correct environment. If this is your first time runnning this notebook/missing some packages, please refer to README.md for more detail on how to set up the environment.

In [ ]:
import numpy as np
import torch
from torch import nn
import os
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from dataset.dataset import ego4dDataset
from dataset.data_vis import *
from utils.loss import Heatmap_Loss
import torchvision.transforms as transforms
from utils.function import *

### 1 - Load Ego4D dataset

To start with, let's load in the Ego-Exo4D data and visualize the ground truth pose annotation. The data we will be using is a subset of the large original Ego-Exo4D dataset, and you can download the annotation JSON files and images from [here](https://drive.google.com/drive/folders/1IWfQIfKzwvSfcBf9G2V18edkkk6SqE6M?usp=sharing). Put the annotation under `anno_dir` and unzipped images under `img_dir` wtih file structure looks like this:


```
{anno_dir}
    ├── ego_pose_gt_anno_test.json
    ├── ego_pose_gt_anno_train.json
    └── ego_pose_gt_anno_val.json
```

```
{img_dir}
    ├── train
    │   ├── cmu_bike01_1
    │   ├── ...
    │
    ├── val
    │   ├── georgiatech_bike_07_10
    │   ├── ...
    │
    └── test
        ├── cmu_bike06_2
        ├── ...
```

*NOTE: Ego-Exo4D dataset annotation is still in progress, thus there might be some bad/missing annotations. However, the general quality of the dataset should be good enough for you to train a hand pose estimation model that gives reasonable predictions.*

In [ ]:
# TODO: Modify config as needed, e.g. annotation and image directory, training batch size etc.
cfg = {
        "anno_dir": ...,
        "img_dir": ...,
        "lr": 1e-1,
        "train_bs": 64,
        "val_bs": 64,
        "epochs": 20,
        "image_dim": 112,
        "heatmap_dim": 112,
        "img_mean": [0.485, 0.456, 0.406],
        "img_std": [0.229, 0.224, 0.225],
    }

# Define the transform for image data preprocessing, which in here is just image normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cfg["img_mean"],
                         std=cfg["img_std"])
])

# TODO: Initialize the train, val and test Dataset
# Hint: take a look at the implementation of ego4dDataset for initialization requirement
train_dataset = ego4dDataset(cfg=...,
                             split=...,
                             transform=...)

val_dataset = ego4dDataset(cfg=...,
                           split=...,
                           transform=...)

test_dataset = ego4dDataset(cfg=...,
                            split=...,
                            transform=...)

# Check the dataset length
print("Train: ", len(train_dataset))
print("Val: ", len(val_dataset))
print("Test: ", len(test_dataset))

**Visualizing the dataset**

With the top-down method, the hand pose estimation task is further defined as a heatmap regression task. Instead of directly predicting the 2D (x,y) coordinates of each hand joints, we represent each 21 hand joints (See Figure 3 for joint index relation) as a heatmap s.t. a higher value means a higher probabilities of a ground truth joint. The goal of the model is to regress each joint position by producing a set of heatmaps, and we can get the predicted (x,y) coordinate of each joint by selecting the point with maximum values in heatmap as shown in Figure 4.

<div style="display: flex; justify-content: space-between; text-align: center;">
    <figure>
        <img src="imgs/hand_index.png" alt="Hand index" width ="300" height="300">
        <figcaption>Figure 3: Hand index</figcaption>
    </figure>
    <figure>
        <img src="imgs/hand_pose_hm.png" alt="Hand pose heatmap" width ="900" height="280">
        <figcaption>Figure 4: Hand pose estimation from heatmap</figcaption>
    </figure>
</div>

In cell below, you can select data from a different dataset and visualize those ground truth poses to get a better understanding about the heatmap.

In [ ]:
## TODO: Modify as needed to take a look at the dataset
check_dataset = train_dataset
idx = ...

## Get one dataset sample for visualization
input, gt_heatmap, gt_kpts_weight, metadata = check_dataset[idx]

## Visualization based on heatmap
# 1. Transform normalized image back to original RGB images in [0,1]
vis_img = inverse_normalize(input, cfg["img_mean"], cfg["img_std"]).permute(1,2,0)
# 2. Find (x,y) of each joint from ground truth heatmap
gt_kpts_from_hm = get_max_preds(gt_heatmap.unsqueeze(0).numpy()).squeeze(0)
# 3. Remove invalid joints by assigned None
gt_kpts_from_hm[~gt_kpts_weight] = None
# Visualization
vis_data(vis_img, gt_kpts_from_hm)

### 2 - Define model

The model we will be using is a modified version of [U-Net](https://arxiv.org/abs/1505.04597) with fewer layers for faster training with reasonable performance. Assume batch_size=1, it takes a RGB image $I (1,3,112,112)$ as input and produce a set of heatmaps $H (1,21,112,112)$ as output. See Figure 5 for model architecture details, where each box corresponds to a multi-channel feature map whose number of channels is denoted on top and spatial dimensions is denoted at the bottom left. If no numbers denoted, then it has the same value as previous one. Each arrow denotes some operations.

<div style="display: center; justify-content: space-between; text-align: center;">
    <figure>
        <img src="imgs/model.png" alt="Model architecture" style="width: 80%;">
        <figcaption>Figure 5: Model architecture</figcaption>
    </figure>
</div>

The model architecture shapes like the English letter U which is why it is called U-Net. `D` is a hyperparameter we could control over the number of expanded channels, which is 16 by default. The feature map first follows a contracting path, where the spatial dimension is reduced x2 by pooling and number of channels is increased by convolutions. Then it follows an expansive path, where the spatial dimension is increased x2 by upsampling and number of channels is reduced by convolutions. The feature maps from earlier layers are also concatenated with feature maps in later layers in expansive path so that the features from eariler layers are reused, which proves to help model convergence and generate better result.

Also notice the repetitive existence of two consecutive blue arrows in the model architecture, which represent two 3x3 convolutions with batch normalization and ReLU operation. They only change the channels of the feature map but keep spatial dimension unchanged. It turns out we can include those operations denoted by these two consecutive blue arrows as a separate module `ConvBlock`, which only takes two parameters `in_ch`and `out_ch` as input to define all the inner layers, with more details about each layer from table below. In this way, we can greatly simplify our model module design by reusing the `ConvBlock` module to avoid repetition.

**ConvBlock:**
|  Operation  |                                Hyperparameters                                |
|:-----------:|:-----------------------------------------------------------------------------:|
|    Conv2d   |  in_channels=in_ch, out_channels=out_ch, kernel_size=3, padding=1, bias=False |
| BatchNorm2d |                              num_features=out_ch                              |
|     ReLU    |                                                                               |
|    Conv2d   | in_channels=out_ch, out_channels=out_ch, kernel_size=3, padding=1, bias=False |
| BatchNorm2d |                              num_features=out_ch                              |
|     ReLU    |                                                                               |

All the pooling/upsampling operation can be performed with the same `torch.nn` module (since there is no trainable weights), see below for their hyperparameters:

| Operation |                    Hyperparameters                   |
|:---------:|:----------------------------------------------------:|
| MaxPool2d |                     kernel_size=2                    |
|  Upsample | scale_factor=2, mode="bilinear", align_corners=False |

The final convolution operations (represented by the pink arrow) is just one 3x3 convolution with sigmoid function:

| Operation |                            Hyperparameters                            |
|:---------:|:---------------------------------------------------------------------:|
|   Conv2d  | in_channels=D, out_channels=21, kernel_size=3, padding=1, bias=False  |
|  Sigmoid  |                                                                       |

With all those information, please fill in the TODOs below to build the entire model architecture.

In [ ]:
class ConvBlock(nn.Module):

    def __init__(self, in_ch, out_ch):
        super().__init__()
        # TODO: Define each layer as shown above
        # Hint: You can use nn.Sequential() to organize all operations together.
        # See https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html#sequential


    def forward(self, x):
        # TODO: Pass input through each layers and return final output
        return ...


class ShallowUNet(nn.Module):
    """
    Model architecture of shallower UNet for 2D hand pose estimation
    """
    def __init__(self, in_channel=3, out_channel=21, MODEL_NEURONS=16):
        super().__init__()
        # TODO: Define each layers and operations in model
        # Hint: reuse ConvBlock

    def forward(self, x):
        # TODO: Pass input through each layers/operations and return final output
        # Hint: to concat two tensors you can use torch.concat()
        return ...

In [ ]:
# Run this test cell to check the model output shape
dummy_input = torch.rand(1,3,112,112)
model = ShallowUNet()
dummy_output = model(dummy_input)
assert dummy_output.shape == (1,21,112,112), "Incorrect model output shape. Please check your model design"

### 3 - Train model

In this part, you need to define the pipeline of model training and validation similar to Project 1. See below TODOs to fill in your implementation.

In [ ]:
def train(train_loader, model, criterion, optimizer, device, output_dir):
    # TODO: set model to training mode


    # Keep track of training loss
    batch_loss = []
    debug_step = len(train_loader) // 6

    train_loader = tqdm(train_loader, dynamic_ncols=True)
    # Iterate over all training samples
    for i, (input, gt_hm, kpts_weight, _) in enumerate(train_loader):
        # TODO:
        # 1. Put all revelant data onto same device
        # 2. Model forward (given cropped hand image, predict a set of heatmaps)
        pred_hm = ...


        # TODO: Compute loss
        loss = ...
        batch_loss.append(loss.item())


        # TODO:
        # 1. Clear the old parameter gradients
        # 2. Compute the derivative of loss w.r.t the model parameters
        # 3. Update the model parameters with optimizer


        # Save debugging images every debug_step
        if (i+1) % debug_step == 0:
            # Save debugging images
            prefix = '{}_{}'.format(os.path.join(output_dir, 'train'), i)
            pred_kpts = get_max_preds(pred_hm.detach().cpu().numpy())
            gt_kpts = get_max_preds(gt_hm.detach().cpu().numpy())
            save_debug_images(input, gt_kpts, gt_hm, pred_kpts, pred_hm, kpts_weight, prefix)
    # Return average training loss
    return np.mean(batch_loss)


def validate(val_loader, model, criterion, device, output_dir):
    # TODO: set model to evaluate mode


    # Keep track of validation loss
    batch_loss = []
    debug_step = len(val_loader) // 6

    with torch.no_grad():
        val_loader = tqdm(val_loader, dynamic_ncols=True)
        # Iterate over all validation samples
        for i, (input, gt_hm, kpts_weight, _) in enumerate(val_loader):
            # TODO:
            # 1. Put all revelant data onto same device
            # 2. Model forward (given cropped hand image, predict a set of heatmaps)
            pred_hm = ...


            # Compute loss
            loss = ...
            batch_loss.append(loss.item())


            # Save debugging images every debug_step
            if (i+1) % debug_step == 0:
                # Save debugging images
                prefix = '{}_{}'.format(os.path.join(output_dir, 'val'), i)
                # Get pred and gt kpts and scale back to image scale
                pred_kpts = get_max_preds(pred_hm.detach().cpu().numpy())
                gt_kpts = get_max_preds(gt_hm.detach().cpu().numpy())
                save_debug_images(input, gt_kpts, gt_hm, pred_kpts, pred_hm, kpts_weight, prefix)
    # Return average validation loss
    return np.mean(batch_loss)

With the training and validation pipeline set up, we can then instantiate our model and define other essential parts for model training including:

- **optimizer:** We will be using stochastic gradient descent as our optimizer, with initial learning rate set at `cfg["lr"]` which is 0.1 by default.

- **loss function (criterion):** We will use `Heatmap_Loss()` as our loss function (the implementation is already provided), which is built based on Dice loss to compute the difference between predicted and ground truth heatmap across each joint to regulate the model learning. See this [post](https://pycad.co/the-difference-between-dice-and-dice-loss/) for visual illustration on how Dice loss works. Please take a look at the `Heatmap_Loss()` implementation to see how to properly call it. Remember to pass `gt_kpts_weight` as third argument to loss function so that for those invalid joints they won't be included in loss calculation, thus doesn't affect the training process.

- **scheduler:** scheduler helps to adjust the learning rate to ensure smoother and better training process. To start with, use `MultiStepLR` scheduler to decay the learning rate by 0.1x at epoch 6 and 12. Feel free to try with [different scheduler](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate) as you needed.

- **dataloaders**: train and val dataloaders to return data in batches.

In [ ]:
# TODO: Instantiate the model and define device for training to use GPU if available
device = ...
model = ...

# TODO: Define criterion
criterion = ...
# TODO: Define optimizer
optimizer = ...
# TODO: Define scheduler
scheduler = ...

# Define train and val dataloader
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=cfg["train_bs"],
    shuffle=True,
    num_workers=4,
    pin_memory=True
)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=cfg["val_bs"],
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [ ]:
# Define output directory (where debugging images and model ckpt will be saved)
output_root = "output"
output_dir = os.path.join(output_root, time.strftime("%Y-%m-%d-%H-%M"))
print("="*10 + f" Training started. Output will be saved at {output_dir} " + "="*10)
os.makedirs(output_dir, exist_ok=True)

# Keep track of each epoch train and val loss
all_train_loss = []
all_val_loss = []

# Start training
best_val_loss = np.inf
for e in range(cfg["epochs"]):
    # Train
    epoch_train_loss = train(train_loader, model, criterion, optimizer, device, output_dir)
    all_train_loss.append(epoch_train_loss)

    # Val
    epoch_val_loss = validate(val_loader, model, criterion, device, output_dir)
    all_val_loss.append(epoch_val_loss)

    # Scheduler step
    if scheduler is not None:
        scheduler.step()

    # Log loss
    print("Epoch: {}/{} \t Train loss: {:.5f} \t Val loss: {:.5f}".format(e, cfg["epochs"], epoch_train_loss, epoch_val_loss))

    # Check for best validation loss
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        # Save best model weight with smallest validation loss
        torch.save(model.state_dict(), os.path.join(output_dir, f"final_state.pth.tar"))
        print(f"Saving model weight with best val_loss={epoch_val_loss:.5f}")
    print()
print("="*10 + f" Training finished. Got best model with val_loss={best_val_loss:.5f} " + "="*10)

### 4 - Inference

In this part we will load in the pretrained model weight you got from previous part, and do the model inference on test set to evaluate its performance. By visualizing the predicted hand pose and ground truth pose, we can get an idea whether the model actually learns to find the location of each hand joint from an image.

Considering the fact that the model is fairly simple and there exists some noisy data samples in the dataset, you are not expected to get a superior model gives excellent predictions for all images. However, for images with clear hand present e.g. hands playing pianos, the model should give reasonably good predictions in general. See one example below on how the model actually predicts a better hand pose compared with the noisy ground truth annotation on one test image.

<div style="display: center; justify-content: space-between; text-align: center;">
    <img src="imgs/model_inference_sample.png" alt="Example of better model prediction than noisy ground truth" width="700" height="360">
</div>


In [ ]:
# TODO: Initialize model and load in pretrained weight
# Hint: use given helper function load_pretrained_weights() to load in model weight
model = ...

In [ ]:
# TODO: Modify index to check model inference result on different images
inference_dataset = test_dataset
idx = ...

# Get one data sample from test set
input, gt_hm, kpts_weight, meta = inference_dataset[idx]
vis_img = inverse_normalize(input, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]).permute(1,2,0)

# GT
gt_kpts = get_max_preds(gt_hm.unsqueeze(0).detach().cpu().numpy())
gt_kpts = gt_kpts.squeeze(0)
gt_kpts[~kpts_weight] = None
vis_data(vis_img, gt_kpts, "gt")

# Prediction
with torch.no_grad():
    pred_hm = model(input.unsqueeze(0))
pred_kpts = get_max_preds(pred_hm.detach().cpu().numpy())
pred_kpts = pred_kpts.squeeze(0)
pred_kpts[~kpts_weight] = None
vis_data(vis_img, pred_kpts, "pred")

### 5 - Reflection

TODO: Discuss as a group and answer the following questions in an new cell below.
1. To what challenges could a solution be developed to address, that incorporates hand pose estimation? You may disucss more than one idea.
2. To what challenges could a solution be developed to address, that incorporates body pose estimation? You may disucss more than one idea.
3. If any, which part(s) of this assignment did you find most challenging to understand and/or implement?

### 6 - Submission

TODO: After you have completed this notebook, submit the following items listed below to Gradescope. Only one group member needs to make a submission, and add the other group member.

Requirements for submission:
1. The completed .ipynb file in .ipynb format (you must run all cells before saving the file for submission).
2. The completed .ipynb file in .pdf format (you must run all cells before saving the file for submission).
3. 3 images from the test dataset visualizing hand-pose estimation after running inference.